In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, db
from sklearn.metrics.pairwise import cosine_similarity

if not firebase_admin._apps:
    
    cred = credentials.Certificate('nope.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sistema-rec-dmi-default-rtdb.firebaseio.com/'
    })
ref = db.reference('/base_usuarios')
ref2 = db.reference('/base_materias')

datos_firebase = ref.get()
datos_firebase2 = ref2.get()

df_firebase = pd.DataFrame(datos_firebase)
df_materias = pd.DataFrame(datos_firebase2)

rating_columns = [
    'electiva',
    'general_rating',
    'method_rating',
    'personalGrowth_rating',
    'rec_rating',
    'teacher_rating',
    'time_rating'
]

prefixes = ['first', 'second', 'third']

df_ratings_list = []

for indice, fila in df_firebase.iterrows():
    determinante = fila['determinante_electivas']
    
    for i, prefix in enumerate(prefixes, start=1):
        electiva_col = f'{prefix}_electiva'
        general_col = f'{prefix}_general_rating'
        method_col = f'{prefix}_method_rating'
        personalGrowth_col = f'{prefix}_personalGrowth_rating'
        rec_col = f'{prefix}_rec_rating'
        teacher_col = f'{prefix}_teacher_rating'
        time_col = f'{prefix}_time_rating'
        
        electiva = fila[electiva_col]
        
        if electiva.lower() != 'no':
            nueva_fila = {
                'electiva': electiva,
                'general_rating': fila[general_col] if fila[general_col] != '' else None,
                'method_rating': fila[method_col] if fila[method_col] != '' else None,
                'personalGrowth_rating': fila[personalGrowth_col] if fila[personalGrowth_col] != '' else None,
                'rec_rating': fila[rec_col] if fila[rec_col] != '' else None,
                'teacher_rating': fila[teacher_col] if fila[teacher_col] != '' else None,
                'time_rating': fila[time_col] if fila[time_col] != '' else None,
            }
            
            df_temporal = pd.DataFrame([nueva_fila], columns=rating_columns)
            df_ratings_list.append(df_temporal)

df_ratings = pd.concat(df_ratings_list, ignore_index=True)
df_ratings = df_ratings.dropna()

def find_similar_electives(electiva_seleccionada, top_n=3):
    selected_elective_data = df_ratings[df_ratings['electiva'] == electiva_seleccionada]
    selected_elective_data = selected_elective_data.drop(columns=['electiva'])
    similarity_scores = cosine_similarity(selected_elective_data, df_ratings.drop(columns=['electiva']))
    similar_indices = similarity_scores.argsort()[0][-top_n - 1:-1][::-1]
    similar_electives = df_ratings.iloc[similar_indices]['electiva'].tolist()

    return similar_electives

electiva_seleccionada = 'Innovación en servicios'
similares = find_similar_electives(electiva_seleccionada)

df_materias = df_materias.rename(columns={'class_name': 'electiva'})

df_completo = pd.merge(df_ratings, df_materias, on='electiva', how='inner')

df_promedios = df_ratings.groupby('electiva').agg({
    'general_rating': 'mean',
    'method_rating': 'mean',
    'personalGrowth_rating': 'mean',
    'rec_rating': 'mean',
    'teacher_rating': 'mean',
    'time_rating': 'mean'
}).reset_index()

df_promedios_filtrado = df_promedios.merge(df_materias, left_on='electiva', right_on='electiva')

print(df_promedios_filtrado.columns)

df_promedios_filtrado_no_duplicates = df_promedios_filtrado.drop_duplicates(subset='electiva', keep='first')

df_promedio_dict = df_promedios_filtrado_no_duplicates.set_index('electiva').to_dict(orient='index')

def convertir_calificacion(calificacion):
    try:
        return float(calificacion)
    except ValueError:
        return 0

def seleccionar_electiva(datos_usuario):
    usuario = {}
    for elemento in datos_usuario['usuario']:
        usuario.update(elemento)

    for clave, valor in usuario.items():
        usuario[clave] = convertir_calificacion(valor)

    variables_interes = ['general_rating', 'method_rating', 'personalGrowth_rating', 'rec_rating', 'teacher_rating', 'time_rating']

    similitudes = {}
    for electiva, perfil in df_promedio_dict.items():
        perfil_usuario = [usuario.get(k, 0) for k in variables_interes]
        perfil_electiva = [perfil.get(k, 0) for k in variables_interes]

        similitud = cosine_similarity([perfil_usuario], [perfil_electiva])[0][0]

        similitudes[electiva] = similitud

    electivas_afines = sorted(similitudes, key=similitudes.get, reverse=True)[:5]
    afinidades = [similitudes[electiva] for electiva in electivas_afines]

    electiva_seleccionada = None
    for electiva in electivas_afines:
        if df_promedio_dict[electiva]['classification'] == usuario['classification']:
            electiva_seleccionada = electiva
            break

    if electiva_seleccionada is None:
        electiva_seleccionada = electivas_afines[0]

    afinidad = similitudes[electiva_seleccionada]

    return electiva_seleccionada

def construir_json(materia):
    materia_info = df_materias[df_materias['electiva'] == materia].iloc[0]

    info = {
        "class_name": materia_info['electiva'],
        "prefer": materia_info['classification'],
        "Horario": materia_info['day'], 
        "modalidad": materia_info['modal'],
        "similares" : find_similar_electives(materia, top_n=3)
    }

    return info


Index(['electiva', 'general_rating', 'method_rating', 'personalGrowth_rating',
       'rec_rating', 'teacher_rating', 'time_rating', 'classification', 'day',
       'modal'],
      dtype='object')


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/rec-dmi', methods=['POST'])
def recibir_datos():
    datos_recibidos = request.json  
    print(datos_recibidos)
    datos_usuario = datos_recibidos
    electiva_seleccionada = seleccionar_electiva(datos_usuario)
    respuesta = construir_json(electiva_seleccionada)
    print(respuesta)
    return respuesta

if __name__ == '__main__':
    app.run(port=5000) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Nov/2023 19:23:46] "OPTIONS /rec-dmi HTTP/1.1" 200 -
127.0.0.1 - - [24/Nov/2023 19:23:46] "POST /rec-dmi HTTP/1.1" 200 -


{'usuario': [{'general_rating': 5}, {'method_rating': '3'}, {'personalGrowth_rating': '3'}, {'rec_rating': '3'}, {'teacher_rating': '3'}, {'time_rating': 5}, {'classification': 'develop'}, {'day': 'martes'}, {'modal': 'presencial'}]}
{'class_name': 'Innovación en modelos de negocio', 'prefer': 'UX', 'Horario': 'miercoles', 'modalidad': 'presencial', 'similares': ['Diseño de experiencia de usuario (UX & UI)', 'Estrategia de innovación', 'De otra carrrera']}
